In [1]:
import requests
import re
import time
import pandas as pd
from skimage import io
from random import randrange
from selenium import webdriver
from selenium.webdriver.common.by import By
import warnings
warnings.simplefilter("ignore")
import matplotlib.pyplot as plt
%matplotlib inline


In [5]:
search_terms = [
    "Men Topwear Formal",
    "Men Topwear Casual",
    "Men Topwear Party",
    "Women Topwear Formal",
    "Women Topwear Casual",
    "Women Topwear Party",
    "Men Bottomwear Formal",
    "Men Bottomwear Casual",
    "Men Bottomwear Party",
    "Women Bottomwear Formal",
    "Men Footwear Party",
    "Women Footwear Formal",
    "Women Bottomwear Casual",
    "Women Bottomwear Party",
    "Men Footwear Formal",
    "Men Footwear Casual",
    "Women Footwear Casual",
    "Women Footwear Party",
]


In [ ]:
product_ids, product_urls, image_urls, descriptions, brands, genders, categories, occasions = ([] for _ in range(8))

# total number of products per page
MAX_PRODUCTS_PER_CATEGORY = 50

for search_term in search_terms:

    for k in range(1,1000):
        # Number of scraped products over page
        total = 0
        # Regenerate the window for each page not to get ( Access Denied ) message for interacting in the same window
        driver = webdriver.Chrome(r'your_driver_path\chromedriver')
        driver.get(f"https://www.myntra.com/{search_term}?p={k+1}")
        while total < MAX_PRODUCTS_PER_CATEGORY:
            j = 0

            # product card
            wholeDiv = driver.find_elements(By.XPATH, '//li[@class="product-base"]')

            if len(wholeDiv) == 0:
                print("No more products found for this category")
                break

            time.sleep(3)
            for i in wholeDiv:

                txt = i.get_property('innerHTML')
                x = re.search('2.6x,', txt)
                y = re.search('" type="image/webp"><img draggable="', txt)

                # if image is not available then skip the product
                if (x == None or y == None):
                    continue

                total += 1
                
                # product_id
                product_ids.append(f"{''.join(list(x[0] for x in search_term))}_{'{0:0>10}'.format(str(j))}")

                # product_url
                txt = i.get_property('innerHTML')
                x = re.search('" target="_blank" href="', txt)
                y = re.search('" style="display: block;"><div class="', txt)

                product_urls.append(txt[x.end():y.start()])

                # image_urls
                txt = i.get_property('innerHTML')
                x = re.search('2.6x,', txt)
                y = re.search('" type="image/webp"><img draggable="', txt)
                image_urls.append(txt[x.end():y.start()][5:-5])
                # download images into /images folder
                img_data = requests.get(txt[x.end():y.start()][5:-5]).content
                with open(f'44/{randrange(10000000)}.jpg', 'wb') as handler:
                    handler.write(img_data)

                # description
                txt = i.get_property('innerHTML')
                x = re.search('" class="img-responsive" alt="', txt)
                y = re.search('" title="', txt)
                descriptions.append(txt[x.end():y.start()])

                # Brand
                txt = i.get_property('innerHTML')
                x = re.search('"><h3 class="product-brand">', txt)
                y = re.search('</h3><h4 class="product-product">', txt)

                brands.append(txt[x.end():y.start()])
                genders.append(search_term.split()[0])
                categories.append(search_term.split()[1])
                occasions.append(search_term.split()[2])

                j += 1
                k += 1
            driver = webdriver.Chrome(r'your_driver_path\chromedriver')
            driver.get(f"https://www.myntra.com/{search_term}?p={k+1}")
        
        


In [17]:
# create a dataframe
df = pd.DataFrame()

# add columns to the dataframe
df['product_id'] = product_ids
df['product_url'] = product_urls
df['image_url'] = image_urls
df['description'] = descriptions
df['brand'] = brands
df['genders'] = genders
df['categories'] = categories
df['type'] = occasions

# save the dataframe to csv file
df.to_csv('myntra.csv', index=False)


In [ ]:
df = pd.read_csv("myntra.csv")
print(f"Total Products: {df.shape[0]}")
df.head()


In [ ]:
genders = list(df.genders.unique())
categories = list(df.categories.unique())
occasions = list(df.type.unique())

plt.figure(figsize=(18, 32), dpi=200)
i = 1
for gender in genders:
    for category in categories:
        for occasion in occasions:
            mdf = df[(df.genders == gender) & (df.categories ==
                                              category) & (df.type == occasion)]
            img_url = mdf.image_url.iloc[0]
            plt.subplot(6, 3, i)
            plt.imshow(io.imread(img_url))
            plt.axis("off")
            plt.title(f"{gender}s {occasion} {category}".title().replace("ns", "n's"))
            i += 1
